In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [2]:
def Normalization(x):
    max_val = max(x)
    min_val = min(x)
    return [(i-min_val)/(max_val-min_val) for i in x]

In [3]:
def weight_lost_cal(real, pre):
    
    test_auc = metrics.roc_auc_score(real, pre)
    fpr, tpr, thresholds = metrics.roc_curve(real, pre, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    
    print("AUC value = " + str(test_auc))
    print("Score = " + str(score))
    return score

In [4]:
# read data
root = "./data/"
data_train = pd.read_csv(root + "atec_anti_fraud_train.csv")
# data_train_add = pd.read_csv(root + "atec_anti_fraud_train_added.csv")
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")

In [5]:
data_test = data_test.drop(['id'], 1)
data_test = data_test.drop(['date'], 1)

data_train = data_train.drop(['id'], 1)
data_train = data_train.drop(['date'], 1)
# data_train = pd.concat([data_train,data_train_add], sort=True)

data_train['label'] = np.abs(data_train['label'])
train_y = data_train['label']
data_train = data_train.drop(['label'], 1)

# 删除特征
b = ['20','22','24','26','28','30','32','34','46','47','48','50','52','53']        
c = [str(i) for i in range(64,72)]
d = [str(i) for i in range(111,155)]

drop_col = ['f'+i for i in (b+c+d)]

data_test = data_test.drop(drop_col,1)
data_train = data_train.drop(drop_col,1)

In [6]:
train_x = data_train
test_x = data_test
# train_y = np.array(train_y).reshape(-1, 1)

# train_x = np.array(train_x)
# train_y = np.array(train_y).reshape(-1, 1)
# test_x = np.array(test_x)

X = np.array(train_x)
Y = np.array(train_y)
X_test = test_x

X_test = test_x

In [ ]:
# k = 5
# kf = KFold(n_splits = k, shuffle=True, random_state=2)
# kf = KFold(n_splits=k)

params_test={
     'max_depth': [5,7],
#      'gamma': [0, 0.01, 0.1], # 如果分裂能够使loss函数减小的值大于gamma，则这个节点才分裂。
     'lambda_l2': [0, 0.1, 1], # L2正则项
     'learning_rate': [0.01, 0.1],
    'num_leaves': [31, 63]
}

    
model_lgb = lgb.LGBMRegressor(
    # max_depth, gamma, subsample+colsample_bytree, 
    boosting_type='gbdt',
    objective='binary', # 二分类问题
        num_leaves=31, # default=31
        learning_rate=0.1, # = learning_rate
        n_estimators=100, # default=100 
         lambda_l2=0,
#         gamma=0.1
        max_depth=5,
    seed=1,
    metric='auc', 
    subsample = 0.8,
    output_result = 'LightGBM_predict_result.txt',
    colsample_bytree = 0.8)

gsearch = GridSearchCV(estimator=model_lgb, 
                        param_grid=params_test, 
                        scoring='roc_auc', 
                        cv=5, 
                        verbose=1)

gsearch.fit(X,Y)





Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [11]:
print('Best parameters found by grid search are:', gsearch.best_params_)
print('Best score is:', gsearch.best_score_)

Best parameters found by grid search are: {'lambda_l2': 0.1, 'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 63}
Best score is: 0.9825989250931323


In [ ]:
result = 0
xgb_result = 0
importances = 0

# 计算结果
result /= k
xgb_result /= k

# 检查结果
print(result)
print(max(xgb_result))
print(min(xgb_result))

In [ ]:
data_test = pd.read_csv(root + "atec_anti_fraud_test_a.csv")
data_test['score'] = xgb_result
data_test['score'].round(decimals=5)
data = data_test[['id', 'score']]  #选择表格中的两列
data.to_csv(root + "result_717_1000.csv", index=None, encoding='utf8')